In [ ]:

#Implement Page Rank Algorithm. (Use python or beautiful soup for implementation)

In [1]:
import networkx as nx  # Importing networkx to handle graph structures

# PageRank algorithm implementation
def pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, nstart=None, weight='weight', dangling=None):
    # If the graph is empty, return an empty dictionary
    if len(G) == 0:
        return {}

    # If the graph is not directed, convert it to a directed graph
    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G
    
    # Create a stochastic graph, which means normalizing the edge weights
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()  # Total number of nodes in the graph
    
    # Choose a starting vector if not provided
    if nstart is None:
        # If no starting vector, distribute rank equally among all nodes
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # If a starting vector is provided, normalize it
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())
    
    # Handle personalization (bias towards certain nodes)
    if personalization is None:
        # If no personalization is given, distribute rank equally among all nodes
        p = dict.fromkeys(W, 1.0 / N)
    else:
        # Ensure personalization vector covers all nodes in the graph
        missing = set(G) - set(personalization)
        if missing:
            raise NetworkXError(f'Personalization dictionary must have a value for every node. Missing nodes {missing}')
        # Normalize the personalization vector
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())
    
    # Handle dangling nodes (nodes with no outgoing edges)
    if dangling is None:
        # If no specific dangling vector, use the personalization vector
        dangling_weights = p
    else:
        # Ensure dangling vector covers all nodes in the graph
        missing = set(G) - set(dangling)
        if missing:
            raise NetworkXError(f'Dangling node dictionary must have a value for every node. Missing nodes {missing}')
        # Normalize the dangling vector
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v / s) for k, v in dangling.items())
    
    # Identify the dangling nodes (nodes with no outgoing edges)
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
    
    # Perform power iteration: this is the main loop of the algorithm
    for _ in range(max_iter):
        xlast = x  # Store the previous iteration's rank vector
        x = dict.fromkeys(xlast.keys(), 0)  # Initialize a new vector for this iteration
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)  # Account for dangling nodes
        
        for n in x:
            # For each node, sum the contributions from its neighbors
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            
            # Add contributions from the dangling nodes and the personalization vector
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]
        
        # Check convergence: if the L1 norm (absolute difference) is smaller than the tolerance, we stop
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N * tol:
            return x  # Return the final PageRank values

    # If the algorithm did not converge within the maximum iterations, raise an error
    raise NetworkXError(f'Pagerank: power iteration failed to converge in {max_iter} iterations.')

# Example usage of the pagerank function
import networkx as nx

# Generate a random graph using Barabási-Albert model (scale-free network)
G = nx.barabasi_albert_graph(60, 41)  # 60 nodes and each new node has 41 edges

# Compute the PageRank using the custom pagerank function
pr = pagerank(G, alpha=0.4)

# Print the resulting PageRank values for each node in the graph
print(pr)

# Alternatively, use NetworkX's built-in PageRank function
pr_builtin = nx.pagerank(G, 0.4)  # 0.4 is the damping factor
print("NetworkX's built-in PageRank results:")
print(pr_builtin)


{0: 0.029220024315633055, 1: 0.013178046038622323, 2: 0.01298395312167649, 3: 0.013568986982610786, 4: 0.012780141376184075, 5: 0.013159264936689043, 6: 0.012574833025542645, 7: 0.01255807050225112, 8: 0.012773473116328848, 9: 0.013162730587953895, 10: 0.010406075051523841, 11: 0.012968611521553833, 12: 0.012551602046972584, 13: 0.012970460556007432, 14: 0.012997821934378382, 15: 0.012568412417128031, 16: 0.012554691754219532, 17: 0.012961998721781245, 18: 0.013179415576964127, 19: 0.012951814191034519, 20: 0.01316680838894304, 21: 0.012953813502760578, 22: 0.013774701730934216, 23: 0.01337983844127945, 24: 0.01274501851295722, 25: 0.01316395859973783, 26: 0.013362879722658887, 27: 0.012952237248468964, 28: 0.013375995620356584, 29: 0.012963992508250512, 30: 0.013186911657565654, 31: 0.013162108162540728, 32: 0.013164807030920992, 33: 0.012770746032099335, 34: 0.013375766165547098, 35: 0.013568986982610786, 36: 0.012764625267483733, 37: 0.012959054845302033, 38: 0.012565248368754167, 3

In [ ]:
# Code Breakdown:

# Imports:
# - networkx: Used to handle graph creation and manipulation.

# pagerank() Function Parameters:
# - G: The input graph (must be a directed graph).
# - alpha: Damping factor (usually set between 0.85 and 0.9).
# - personalization: A dictionary that assigns custom rank values to nodes (used to bias the algorithm towards specific nodes).
# - max_iter: The maximum number of iterations the algorithm will run before stopping.
# - tol: The tolerance for convergence. If the sum of absolute differences between iterations is smaller than this value, the algorithm will stop.
# - nstart: A dictionary that initializes the starting PageRank values (optional).
# - weight: The weight of the edges in the graph (default is 'weight').
# - dangling: A dictionary to handle dangling nodes, or nodes with no outgoing edges.

# Handling the Graph:
# - If the graph is not directed, it is converted to a directed graph.
# - A stochastic graph (W) is created, where each edge is normalized to form a valid transition probability.

# Initial Vector:
# - The initial PageRank vector (x) is set either as equal distribution (if nstart is None) or from the given starting vector nstart.

# Personalization:
# - If personalization is not provided, the rank is evenly distributed across all nodes.
# - If provided, it is normalized to sum to 1.

# Dangling Nodes:
# - A "dangling node" has no outgoing edges. The algorithm handles these by redistributing their rank using the personalization vector (or a provided dangling vector).

# Power Iteration:
# - The main iterative loop performs the power iteration process to update the PageRank of each node.
# - The rank is updated based on the previous iteration's rank and contributions from neighbors and dangling nodes.
# - The algorithm stops when the L1 norm of the change in ranks is below the specified tolerance, or when the maximum number of iterations is reached.

# Return:
# - The function returns the computed PageRank values if convergence is reached.
# - Otherwise, it raises an error if convergence isn't achieved within the given iterations.

# Example Use Case:
# - A random graph is created using the Barabási-Albert model (nx.barabasi_albert_graph()), which generates a scale-free network.
# - The pagerank() function is applied to this graph with a damping factor (alpha=0.4), and the resulting PageRank values are printed.
# - The NetworkX built-in PageRank function is also used to verify the output.

# Output:
# - The pagerank() function will output a dictionary of nodes with their corresponding PageRank values.
# - This can be useful for ranking nodes (pages) in terms of importance or centrality within a network.
